In [1]:
import json
import os
os.chdir("..")

In [2]:
import pprint

In [3]:
from IO.constants import Constants
from profev.Car import Car
from profev.ChargingStation import ChargingStation
from profev.CarPark import CarPark

In [4]:
# Inputs

path = os.path.join(os.getcwd(), "utils", "ae5dc1e61acd", "Input.registry.file")
with open(path, "r") as file:
    input_config_file = json.loads(file.read())

input_config_mqtt = {}
model_name = "CarParkModel"

### Helper functions

In [5]:

def generate_charger_classes(chargers):
    chargers_list = []
    for charger_name, charger_detail in chargers.items():
        max_charging_power_kw = charger_detail.get("Max_Charging_Power_kW", None)
        hosted_car = charger_detail.get("Hosted_Car", None)
        soc = charger_detail.get("SoC", None)
        assert max_charging_power_kw, f"Incorrect input: Max_Charging_Power_kW missing for charger: {charger_name}"
        chargers_list.append(ChargingStation(max_charging_power_kw, hosted_car, soc))
    return chargers_list


def generate_car_classes(cars):
    cars_list = []
    for car_name, car_detail in cars.items():
        battery_capacity = car_detail.get("Battery_Capacity_kWh", None)
        assert battery_capacity, f"Incorrect input: Battery_Capacity_kWh missing for car: {car_name}"
        cars_list.append(Car(car_name, battery_capacity))
    return cars_list

## Input parser

In [30]:
from functools import partial
from profev.MonteCarloSimulator import simulate

class InputConfigParser:
    
    def __init__(self, input_config_file, input_config_mqtt, model_name):
        self.model_name = model_name
        self.model_variables = {'Feasible_ESS_Decisions': {'type': 'Set', 'indexing': 'None'}, 'Feasible_VAC_Decisions': {'type': 'Set', 'indexing': 'None'}, 'Value_Index': {'type': 'Set', 'indexing': 'None'}, 'Value': {'type': 'Param', 'indexing': 'index'}, 'P_PV_Forecast': {'type': 'Param', 'indexing': 'None'}, 'Initial_ESS_SoC': {'type': 'Param', 'indexing': 'index'}, 'Initial_VAC_SoC': {'type': 'Param', 'indexing': 'index'}, 'Number_of_Parked_Cars': {'type': 'Param', 'indexing': 'None'}, 'Unit_Consumption_Assumption': {'type': 'Param', 'indexing': 'None'}, 'Unit_Drop_Penalty': {'type': 'Param', 'indexing': 'None'}, 'ESS_Capacity': {'type': 'Param', 'indexing': 'None'}, 'VAC_Capacity': {'type': 'Param', 'indexing': 'None'}, 'Behavior_Model_Index': {'type': 'Set', 'indexing': 'None'}, 'Behavior_Model': {'type': 'Param', 'indexing': 'None'}, 'dT': {'type': 'Param', 'indexing': 'None'}, 'Decision': {'type': 'Var', 'indexing': 'index'}, 'P_ESS': {'type': 'Var', 'indexing': 'None'}, 'P_VAC': {'type': 'Var', 'indexing': 'None'}, 'P_PV': {'type': 'Var', 'indexing': 'index'}, 'P_GRID': {'type': 'Var', 'indexing': 'None'}}
        self.input_config_file = input_config_file
        self.input_config_mqtt = input_config_mqtt
        self.mqtt_params = {}
        self.generic_names = []
        self.generic_file_names = []
        
        self.defined_prediction_names = []
        self.defined_non_prediction_names = ["P_PV", "P_PV_Forecast"] #New
        self.defined_external_names = ["SoC_Value"]
        self.prediction_names = []
        self.non_prediction_names = []
        self.external_names = []
        
        # Added
        self.car_park = None
        self.simulator = None
        
        self.optimization_params = self.extract_optimization_values()
        
    def add_name_to_list(self, key):
        if key in self.defined_prediction_names:
            self.prediction_names.append(key)
        elif key in self.defined_non_prediction_names:
            self.non_prediction_names.append(key)
        elif key in self.defined_external_names:
            self.external_names.append(key)
        else:
            self.generic_names.append(key)

    def generate_car_park(self, details):
        chargers = details.get("Charging_Station", None)
        cars = details.get("Cars", None)
        assert chargers, "Incorrect input: Charging_Station missing in CarPark"
        assert cars, "Incorrect input: Cars missing in CarPark"
        chargers = dict(chargers)
        cars = dict(cars)
        chargers_list = generate_charger_classes(chargers)
        cars_list = generate_car_classes(cars)
        self.car_park = CarPark(chargers_list, cars_list)

        return self.car_park.number_of_cars, self.car_park.vac_capacity
    
    def generate_behaviour_model(self, plugged_time, unplugged_time, simulation_repetition):
        plugged_time_mean = plugged_time.get("mean", None)
        plugged_time_std = plugged_time.get("std", None)
        
        assert plugged_time_mean, "mean value missing in Plugged_Time"
        assert plugged_time_std, "std value missing in Plugged_Time"
        
        unplugged_time_mean = plugged_time.get("mean", None)
        unplugged_time_std = plugged_time.get("std", None)
        
        assert unplugged_time_mean, "mean value missing in Unlugged_Time"
        assert unplugged_time_std, "std value missing in Unlugged_Time"
        
        self.simulator = partial(simulate,
                                 repetition=simulation_repetition,
                                 unplugged_mean=unplugged_time_mean, unplugged_std=unplugged_time_std,
                                 plugged_mean=plugged_time_mean, plugged_std=plugged_time_std)

    def extract_optimization_values(self):
        data = {}
        for input_config in [self.input_config_file, self.input_config_mqtt]:
            for k, v in input_config.items():
                if isinstance(v, dict):
                    for k1, v1 in v.items():
                        if k1 == Constants.meta:
                            for k2, v2 in v1.items():
                                try:
                                    v2 = float(v2)
                                except ValueError:
                                    pass
                                if isinstance(v2, float) and v2.is_integer():
                                    v2 = int(v2)
                                if k2 in self.model_variables.keys():
                                    indexing = self.model_variables[k2]["indexing"]
                                    if indexing == "index":
                                        data[k2] = {int(0): v2}
                                    elif indexing == "None":
                                        data[k2] = {None: v2}
                                else:
                                    data[k2] = {None: v2}
                        elif k1 == Constants.SoC_Value and isinstance(v1, int):
                            indexing = self.model_variables[Constants.SoC_Value]["indexing"]
                            if indexing == "index":
                                data[Constants.SoC_Value] = {int(0): float(v1 / 100)}
                            elif indexing == "None":
                                data[Constants.SoC_Value] = {None: float(v1 / 100)}
                        elif isinstance(v1, list):
                            self.add_name_to_list(k1)
                        elif k == "generic" and not isinstance(v1, dict):
                            logger.debug("Generic single value")
                            try:
                                v1 = float(v1)
                            except ValueError:
                                pass
                            if isinstance(v1, float) and v1.is_integer():
                                v1 = int(v1)
                            data[k1] = {None: v1}
                        elif k == "PROFEV":
                            if isinstance(v1, dict):
                                if k1 == Constants.CarPark:
                                    number_of_cars, vac_capacity = self.generate_car_park(v1)

                                    data["Number_of_Parked_Cars"] = {None: number_of_cars}
                                    data["VAC_Capacity"] = {None: vac_capacity}

                                if k1 == Constants.Uncertainty:
                                    plugged_time = v1.get("Plugged_Time", None)
                                    unplugged_time = v1.get("Unplugged_Time", None)
                                    simulation_repetition = v1.get("simulation_repetition", None)
                                    
                                    assert plugged_time, "Plugged_Time is missing in Uncertainty"
                                    assert unplugged_time, "Unplugged_Time is missing in Uncertainty"
                                    assert simulation_repetition, "simulation_repetition is missing in Uncertainty"
                                    
                                    self.generate_behaviour_model(plugged_time, unplugged_time, simulation_repetition)
                                    
                                    data["Value"] = "null"
                                    data["Initial_ESS_SoC"] = "null"
                                    data["Initial_VAC_SoC"] = "null"
                                    data["Behavior_Model"] = "null"
                            else:
                                try:
                                    v1 = float(v1)
                                except ValueError:
                                    pass
                                if isinstance(v1, float) and v1.is_integer():
                                    v1 = int(v1)
                                data[k1] = {None: v1}
        #         pprint.pprint(data, indent=4)
        return data
    
icp = InputConfigParser(input_config_file, input_config_mqtt, model_name)

{'repetition': 10000, 'unplugged_mean': 18.76, 'unplugged_std': 1.3, 'plugged_mean': 18.76, 'plugged_std': 1.3}


In [31]:
icp.simulator(time_resolution=3600, horizon=24, max_number_of_cars=7)

{0: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0.9999999999999062},
 1: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0.9999999999999062},
 2: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0.9999999999999062},
 3: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0.9999999999999062},
 4: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0.9999999999999062},
 5: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0.9999999999999062},
 6: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0.9999999999999062},
 7: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0.9999999999999062},
 8: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0.9999999999999062},
 9: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0.9999999999999062},
 10: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0.9999999999999062},
 11: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0.9999999999999062},
 12: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0.9999999999999062},
 13: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0.9999999999999062},
 1